<a href="https://colab.research.google.com/github/Ronin1289/Price-Predictor/blob/main/Price_Predictor_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using TCN

##Importing Libraries

In [1]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2b020544d8ffa0a48130f8551004b81bcfe4246f7ac0aa47af40ccf15272cdc9
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [8]:
!pip install keras-tcn


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import ta
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint

##Loading Data

In [5]:

# Load Data Function
def load_data(ticker, start, end, filename="BTC-USD_data.csv"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=False)
    df['RSI'] = ta.momentum.RSIIndicator(pd.Series(df['Close'].squeeze()), window=14).rsi()
    df['RSI'] = df['RSI'].astype(float)
    df['Trend'] = df['Close'].pct_change().fillna(0)
    df.dropna(inplace=True)
    df.to_csv(filename, index=True)
    print(f"Data saved to {filename}")
    return df

# Load Data
data = load_data("BTC-USD", start="2020-01-01", end="2024-01-01", filename="BTC-USD_data.csv")


[*********************100%***********************]  1 of 1 completed

Data saved to BTC-USD_data.csv


##Preparing Data

In [4]:

# Select Features and Target
feature_columns = ['Close', 'RSI', 'Trend']
target_column = 'Close'

# Convert Data to NumPy Array
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[feature_columns])

# Create Sequences for Time Series
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])  # Predicting Close Price
    return np.array(X), np.array(y)

seq_length = 100  # Past 50 days for prediction
X, y = create_sequences(data_scaled, seq_length)

# Split Data into Train/Test
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN
import ta  # Technical Analysis library
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# ----- Load Data -----
def load_data(ticker, start, end, filename="BTC-USD_data.csv"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=False)  # Ensure raw price data
    df['RSI'] = ta.momentum.RSIIndicator(pd.Series(df['Close'].squeeze()), window=14).rsi()  # Compute RSI
    df['RSI'] = df['RSI'].astype(float)  # Convert to 1D Series if needed
    df['Trend'] = df['Close'].pct_change().fillna(0)  # Compute trend
    df.dropna(inplace=True)  # Remove missing values

    # Save to CSV
    df.to_csv(filename, index=True)
    print(f"Data saved to {filename}")

    return df

# Load and save data
data = load_data("BTC-USD", start="2020-01-01", end="2024-01-01", filename="BTC-USD_data.csv")

# ----- Feature Selection -----
feature_columns = ['Close', 'RSI', 'Trend']
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[feature_columns])

# ----- Prepare Sequences for Time-Series -----
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])  # Predicting "Close" price
    return np.array(X), np.array(y)

SEQ_LENGTH = 70  # Define sequence length
X, y = create_sequences(data_scaled, SEQ_LENGTH)

[*********************100%***********************]  1 of 1 completed

Data saved to BTC-USD_data.csv


###TCN 1 Layer

In [11]:

# ----- Split Data -----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ----- Define TCN Model -----
model = Sequential([
    TCN(input_shape=(X_train.shape[1], X_train.shape[2]),
        nb_filters=64,
        kernel_size=3,
        dilations=[1, 2, 4, 8],
        dropout_rate=0.2),
    Dense(1)  # Output layer
])

# Compile the Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print Model Summary
model.summary()

# ----- Train the Model -----
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    batch_size=32)


/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:273: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ tcn_1 (TCN)                          │ (None, 64)                  │          87,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 87,425 (341.50 KB)

 Trainable params: 87,425 (341.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - loss: 2.7938 - mae: 1.2269 - val_loss: 0.0195 - val_mae: 0.1069
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - loss: 0.2128 - mae: 0.3559 - val_loss: 0.0193 - val_mae: 0.1134
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.1006 - mae: 0.2499 - val_loss: 0.0230 - val_mae: 0.1335
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0678 - mae: 0.2059 - val_loss: 0.0076 - val_mae: 0.0719
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0616 - mae: 0.1911 - val_loss: 0.0056 - val_mae: 0.0627
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 0.0426 - mae: 0.1590 - val_loss: 0.0029 - val_mae: 0.0439
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.0348 - mae: 0.1452 - val_loss: 0.0025 - val_mae: 0.0394
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0289 - mae: 0.1339 - val_loss: 0.0034 - val_mae: 0.0460
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.026

###TCN 2 Layers

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tcn import TCN

# Define the Enhanced TCN Model
model = Sequential([
    TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dropout(0.2),

    TCN(nb_filters=256, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print Model Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:273: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ tcn_5 (TCN)                          │ (None, 70, 64)              │          87,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 70, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 70, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ tcn_6 (TCN)                          │ (None, 256)                 │       1,444,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,551,297 (5.92 MB)

 Trainable params: 1,550,657 (5.92 MB)

 Non-trainable params: 640 (2.50 KB)

In [17]:

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 797ms/step - loss: 0.8268 - mae: 0.7006 - val_loss: 0.2121 - val_mae: 0.4458
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 40s 770ms/step - loss: 0.2804 - mae: 0.4170 - val_loss: 0.0483 - val_mae: 0.1732
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 777ms/step - loss: 0.1614 - mae: 0.3193 - val_loss: 0.0439 - val_mae: 0.1717
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 774ms/step - loss: 0.1243 - mae: 0.2692 - val_loss: 0.0331 - val_mae: 0.1783
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 777ms/step - loss: 0.0824 - mae: 0.2220 - val_loss: 0.0499 - val_mae: 0.2122
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 792ms/step - loss: 0.0688 - mae: 0.2002 - val_loss: 0.1053 - val_mae: 0.3197
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 40s 775ms/step - loss: 0.0604 - mae: 0.1901 - val_loss: 0.1043 - val_mae: 0.3194
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 777ms/step - loss: 0.0449 - mae: 0.1681 - val_loss: 0.1281 - val_mae: 0.3543
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 27s 777ms/

In [ ]:
from sklearn.metrics import r2_score

# Make Predictions on Test Data
y_pred = model.predict(X_test)

# Inverse Transform to Original Scale (if using MinMaxScaler)
y_test_inv = scaler.inverse_transform(np.column_stack((y_test, np.zeros((y_test.shape[0], 2)))))[:, 0]
y_pred_inv = scaler.inverse_transform(np.column_stack((y_pred.flatten(), np.zeros((y_pred.shape[0], 2)))))[:, 0]

# Compute R² Score
r2 = r2_score(y_test_inv, y_pred_inv)
print(f"R² Score: {r2:.4f}")


###TCN 3 Layers

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tcn import TCN

# Define the Enhanced TCN Model
model = Sequential([
    TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dropout(0.2),

    TCN(nb_filters=128, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2, return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),

    TCN(nb_filters=256, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print Model Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:273: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ tcn_2 (TCN)                          │ (None, 70, 64)              │          87,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 70, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 70, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ tcn_3 (TCN)                          │ (None, 70, 128)             │         377,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 70, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 70, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ tcn_4 (TCN)                          │ (None, 256)                 │       1,509,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,995,329 (7.61 MB)

 Trainable params: 1,994,433 (7.61 MB)

 Non-trainable params: 896 (3.50 KB)

In [14]:

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 0.6962 - mae: 0.6349 - val_loss: 46.4149 - val_mae: 6.7723
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 934ms/step - loss: 0.2105 - mae: 0.3393 - val_loss: 0.1348 - val_mae: 0.3586
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 973ms/step - loss: 0.1336 - mae: 0.2814 - val_loss: 0.0170 - val_mae: 0.1087
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 937ms/step - loss: 0.0985 - mae: 0.2418 - val_loss: 0.0545 - val_mae: 0.2301
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 34s 972ms/step - loss: 0.0800 - mae: 0.2179 - val_loss: 0.0326 - val_mae: 0.1538
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 973ms/step - loss: 0.0628 - mae: 0.1968 - val_loss: 0.0908 - val_mae: 0.2981
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 955ms/step - loss: 0.0572 - mae: 0.1867 - val_loss: 0.0821 - val_mae: 0.2822
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 34s 966ms/step - loss: 0.0485 - mae: 0.1709 - val_loss: 0.1360 - val_mae: 0.3648
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step 

##Analyzing Performance

In [15]:
from sklearn.metrics import r2_score

# Make Predictions on Test Data
y_pred = model.predict(X_test)

# Inverse Transform to Original Scale (if using MinMaxScaler)
y_test_inv = scaler.inverse_transform(np.column_stack((y_test, np.zeros((y_test.shape[0], 2)))))[:, 0]
y_pred_inv = scaler.inverse_transform(np.column_stack((y_pred.flatten(), np.zeros((y_pred.shape[0], 2)))))[:, 0]

# Compute R² Score
r2 = r2_score(y_test_inv, y_pred_inv)
print(f"R² Score: {r2:.4f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 367ms/step
R² Score: -11.0550
